In [1]:
import logging
import geopandas as gpd
# from sliderule import icesat2
from sliderule import sliderule, icesat2
from mpl_toolkits.basemap import Basemap
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import geopandas as gpd
from sliderule import gedi, earthdata, raster
import matplotlib.pyplot as plt
from matplotlib import colormaps
import numpy as np
import pandas as pd

import matplotlib.cm as cm
import matplotlib.colors as mcolors


import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
import pandas as pd
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import rasterio
from pyproj import Transformer



from pptx.util import Inches
import matplotlib.pyplot as plt
import os
from pptx import Presentation

In [2]:
region_name='Rift'

In [3]:
#UTM region

utmXmin =575900  # Easting (meters east of central meridian)
utmYmin = 7671600 # Northing (meters north of equator)
utmXmax = 576600
utmYmax = 7672800


# Convert UTM to lat/lon using pyproj with UTM Zone 22N for Jakobshavn Isbræ region
transformer_jakobshavn = Transformer.from_crs("EPSG:32622",  "EPSG:4326", always_xy=True)

# Transform bounding box corners
lon_min, lat_min = transformer_jakobshavn.transform(utmXmin, utmYmin)
lon_max, lat_max = transformer_jakobshavn.transform(utmXmax, utmYmax)

# # Output the range
# lat_min, lat_max, lon_min, lon_max

lon_up = lon_max
lat_up = lat_max
lon_lo = lon_min
lat_lo = lat_min

# Configure Region of Interest
region = [ {"lon":lon_up, "lat": lat_up},
           {"lon":lon_lo, "lat": lat_up},
           {"lon":lon_lo, "lat": lat_lo},
           {"lon":lon_up, "lat": lat_lo},
           {"lon":lon_up, "lat": lat_up} ]

region

[{'lon': -49.07090265692934, 'lat': 69.15454479863793},
 {'lon': -49.08946060367382, 'lat': 69.15454479863793},
 {'lon': -49.08946060367382, 'lat': 69.14398566607677},
 {'lon': -49.07090265692934, 'lat': 69.14398566607677},
 {'lon': -49.07090265692934, 'lat': 69.15454479863793}]

In [ ]:
# 69.096 ​N, 49.373 ​W)

In [ ]:
def plot_elevation_photon1(subset, gdf03x, cnf_colors, length, res,prs):
    gt_value = subset['gt'].unique()[0]

    # Create and save the plot
    fig, ax = plt.subplots(figsize=(10, 6))
    # Plot ATL03 photons with color mapped to yapc_score using global cnf_colors
    for cnf_val in sorted(gdf03x['yapc_score'].dropna().unique()):
        color = cnf_colors.get(cnf_val, 'black')
        label = f"yapc_score={cnf_val}" if cnf_val % 50 == 0 else None  # Optional sparse labeling
        ax.scatter(
            gdf03x[gdf03x['yapc_score'] == cnf_val]['x_atc'],
            gdf03x[gdf03x['yapc_score'] == cnf_val]['height'],
            s=0.5, color=color, alpha=0.6, label=label
        )
    ax.scatter(subset['x_atc'], subset['h_mean'], s=0.8, color='red',
               label=f"ATL06 spot={subset['spot'].unique()[0]}")
    ax.set_xlabel("x_atc")
    ax.set_ylabel("h_mean (m)")
    # ax.set_title(f"GT={gt_value}, RGT={subset['rgt'].unique()[0]}, Cycle={subset['cycle'].unique()[0]}")
    ax.set_title(f"ATL06 h_mean and ATL03 photons for strong beam length={length},res={res},cycle={subset['cycle'].unique()[0]},GT={gt_value}, RGT={subset['rgt'].unique()[0]}, spot={subset['spot'].unique()[0]}")    
    ax.set_ylim([680, 840])
    ax.grid(True)

    ax.legend(markerscale=4)
    plt.tight_layout()

    plot_filename = f"gt_{gt_value}_plot.png"
    fig.savefig(plot_filename, dpi=300, bbox_inches='tight')
    plt.close(fig)

    # Add slide to existing presentation
    slide = prs.slides.add_slide(prs.slide_layouts[5])
    slide.shapes.add_picture(plot_filename, Inches(1), Inches(1), width=Inches(8))

    os.remove(plot_filename)


In [4]:
# Configure ICESat-2 API
icesat2.init("slideruleearth.io")


# Define the global colormap mapping from 0 to 255
norm = mcolors.Normalize(vmin=0, vmax=255)
cmap = cm.get_cmap('viridis')

# Precompute color for each yapc_score
cnf_colors = {val: cmap(norm(val)) for val in range(256)}

/tmp/ipykernel_129/4013872443.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('viridis')


In [7]:
# parms = {
#     "poly": region,
#     "srt": 3,       # land ice
#     "cnf": 4,
#     "ats": 20.0,    # 20m along-track spacing
#     "cnt": 10,
#     "len": 10.0,
#     "res": 20.0
# }
# gdf6 = icesat2.atl06p(parms)
# gdf6 
# Request ATL03 data
gdf03x4 = sliderule.run("atl03x", {
    "poly": region,
    "rgt": 666,
    "srt": 3,
    "cnf": -1,
    "cycle": 3,
    "yapc":{"score":0, "version": 0}# wrap cycle in a list
    #"yapc":{"score":0, "knn": 4}# wrap cycle in a list
})

gdf03x4
# gdf03x4["yapc_score"].unique()

# # # count_night = (gdf03x4["solar_elevation"] < 0).sum()
# # # print(f"Number of nighttime records: {count_night}")

# gdf03x4 

,region,gt,spacecraft_velocity,solar_elevation,yapc_score,rgt,ph_index,height,spot,x_atc,y_atc,cycle,atl03_cnf,background_rate,quality_ph,geometry
time_ns,,,,,,,,,,,,,,,,
2019-05-11 21:11:17.148237568,5,50,7100.380859,16.939684,22,666,19031934,817.571777,5,1.235863e+07,-3188.056641,3,0,2843419.5,0,POINT (-49.08014 69.15461)
2019-05-11 21:11:17.148237568,5,50,7100.380859,16.939684,153,666,19031935,777.341736,5,1.235863e+07,-3188.280029,3,2,2843419.5,0,POINT (-49.08014 69.15460)
2019-05-11 21:11:17.148237568,5,50,7100.380859,16.939684,161,666,19031936,781.117737,5,1.235863e+07,-3188.259033,3,2,2843419.5,0,POINT (-49.08014 69.15460)
2019-05-11 21:11:17.148237568,5,50,7100.380859,16.939684,151,666,19031937,777.726196,5,1.235863e+07,-3188.277832,3,2,2843419.5,0,POINT (-49.08014 69.15460)
2019-05-11 21:11:17.148237568,5,50,7100.380859,16.939684,138,666,19031938,772.977112,5,1.235863e+07,-3188.304199,3,2,2843419.5,0,POINT (-49.08014 69.15460)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-05-11 21:11:17.661537536,5,60,7100.392578,16.939842,17,666,6848611,703.241943,6,1.235983e+07,-3277.640381,3,0,2748100.5,0,POINT (-49.08598 69.14403)
2019-05-11 21:11:17.661537536,5,60,7100.392578,16.939842,13,666,6848612,698.735718,6,1.235983e+07,-3277.666260,3,0,2748100.5,0,POINT (-49.08598 69.14403)
2019-05-11 21:11:17.661637632,5,60,7100.392578,16.939842,17,666,6848613,767.250671,6,1.235983e+07,-3277.271484,3,0,2748100.5,0,POINT (-49.08597 69.14402)


In [ ]:
# subset = gdf6[
#     (gdf6['gt'] == 50) &
#     (gdf6['rgt'] == 666) &(gdf6['spot'] == 5)&(gdf6['cycle'] == 3)

#   ]
# subset

# lengths =  [40.0, 30.0, 20.0]
# resolutions = [20.0, 10.0, 5.0]
# cycles = [3, 9, 13, 14, 25]#for ATL06 for gt=50,rgt=666,spot=5
# # cycle= 3 4 8 9 13 14 15 19 20 24 25 #for gt=50,spot=5 strong beam
# #cycle=[ 3,  9, 13, 14, 25]#for ATL06 for gt=50,rgt=666,spot=5

In [ ]:
lengths =  [40.0, 30.0]
resolutions = [20.0, 10.0, 5.0]
cycles = [3, 9, 13, 14, 25]#for ATL06 for gt=50,rgt=666,spot=5


# Create or open a PowerPoint presentation
pptx_filename = "/home/jovyan/Glacier_preprocessing/Greenland/latestupdatedcode/ATL06_yapcscore_Greenland.pptx"
prs = Presentation(pptx_filename)

for cycle in cycles:
    for res in resolutions:
        for length in lengths:
            # Build ATL06 Request
            parms = {
                "poly": region,
                "srt": 3,       # land ice
                "cnf": -1,#4,-1 unspecified ,3,4:obtain fewer false-positive returns
                "ats": 20.0,    # 20m along-track spacing
                "cnt": 10,
                "len": length,
                "res": res
            }

            gdf6 = icesat2.atl06p(parms)

            # Request ATL03 data
            gdf03x4 = sliderule.run("atl03x", {
                "poly": region,
                "rgt": 666,
                "srt": 3,
                "cnf": -1,
                "cycle": cycle,  # wrap cycle in a list
                "yapc":{"score":0, "version": 0}# wrap cycle in a list
                # "yapc":{"score":0,"knn": 0},# wrap cycle in a list
            })

            # Reproject ATL06 to EPSG:4326 then to EPSG:32622
            gdf6 = gdf6.to_crs(epsg=4326)
            gdf6 = gdf6.to_crs(epsg=32622)

            # Subset ATL06 cand ATL03
            subset = gdf6[
                (gdf6['gt'] == 50) &
                (gdf6['rgt'] == 666) &
                (gdf6['spot'] == 5) &
                (gdf6['cycle'] == cycle)
            ]

            gdf03x = gdf03x4[
                (gdf03x4['gt'] == 50) &
                (gdf03x4['spot'] == 5)
            ]

            
            # count_night = (gdf03x["solar_elevation"] < 0).sum()
            # print(f"Number of nighttime records: {count_night},cycle:{cycle}")

            if not subset.empty and not gdf03x.empty:
                data_pairs = [(subset, gdf03x)]
                for atl06_subset, gdf03x_photons in data_pairs:
                    plot_elevation_photon1(atl06_subset, gdf03x_photons, cnf_colors, length, res, prs)

prs.save(pptx_filename)


[ICESat-2 ATLAS granule: ATL03_20190511210825_06660305_002_02.h5] [RGT 666 gt3l (strong beam), collected 2019-May-11 ​at 21:08 GMT ​= ​19:08 local time, day-time data].

time
2024-07-30 17:16:31.179126528    2.153127e+06
dtype: float64 time
2024-07-30 17:16:31.179126528    652570.518308
dtype: float64

[ICESat-2 ATLAS granule: ATL03_20190511210825_06660305_002_02.h5] [RGT 666 gt3l (strong beam), collected 2019-May-11 ​at 21:08 GMT ​= ​19:08 local time, day-time data].

time
2024-07-30 17:16:31.179126528    2.153127e+06
dtype: float64 time
2024-07-30 17:16:31.179126528    652570.518308
dtype: float64

gdf03x4 = sliderule.run("atl03x", {
    "poly": region,
    "rgt": 666,
    "srt": 3,
    "cnf": -1,
    "cycle": 3,
    "yapc":{"score":0, "version": 0}
})

The only thing I changed is the “yapc” settings to use the value provided in the ATL03 granule.  By setting “cnf” to -1, it effectively grabs all photons that aren’t TEP, so there is no filtering.  By setting the “yapc.score” to 0, it is similarly grabbing all photons with a yapc score of 0 or more, which again means there is no filtering happening.
 
Then in the output, you should have a column called “yapc_score” which is the YAPC classification for that photon (0 to 255).  You will also have a column called “atl03_cnf” which is the confidence classification for the photon; but that comes by default and if you don’t need it, you can just ignore it.